In [2]:
#Lecture 4 - CNN

import tensorflow as tf

In [3]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [4]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import logging

In [6]:
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [7]:
#Tensorflow dataset에서 Fashion mnist를 가져옴! 데이터 접근은 dataset으로,
#데이터의 개수 등 정보는 metadata로 접근 가능함
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)

    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: No module named 'tensorflow_core.estimator'
    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: No module named 'tensorflow_core.estimator'


In [8]:
#train_dataset, test_dataset이라는 배열 모음을 dataset에서 떼와서 따로 정의함
train_dataset, test_dataset = dataset['train'], dataset['test']

In [9]:
#output 번호만 0~9로 저장되어 있고, 이름은 저장되어 있지 않으므로 이렇게 따로 정의
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot']

In [10]:
#훈련 데이터 개수, 테스트 데이터 개수
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("Number of training examples : {}".format(num_train_examples))
print("Number of test examples : {}".format(num_test_examples))

Number of training examples : 60000
Number of test examples : 10000


In [11]:
#데이터 normalize -> 각 28x28픽셀은 0~255 사이 값인데, 이걸 0~1 값으로 바꿈

def normalize(images, labels):
    images = tf.cast(images, tf.float32)
    images /= 255
    return images, labels

#map = train_dataset 내부의 모든 아이템에 normalize 함수를 적용시키는 역할!
train_dataset = train_dataset.map(normalize)
test_dataset = test_dataset.map(normalize)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: No module named 'tensorflow_core.estimator'


In [12]:
train_dataset = train_dataset.cache()
test_dataset = test_dataset.cache()

In [ ]:
#모델 만들기 - layer 제작

model = tf.keras.Sequential([
    #첫 번째 = Conv2D 레이어: (3x3)커널으로 Convolution 진행, padding으로 원본 크기와
    #같은 크기의 이미지를 반환하고, 총 32번 진행함. *relu사용
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                          input_shape=(28, 28, 1)),
    #MaxPooling 레이어: (2x2) 그리드 내부 최댓값 반환, stride=2이므로 원본 크기에서
    #절반으로 준 크기의 이미지를 반환함
    tf.keras.layers.MaxPooling2D((2,2), strides=2),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2,2), strides=2),
    #Convolution 거친 이미지 픽셀 2차원 배열을 1차원 배열로 바꿈
    tf.keras.layers.Flatten(),
    #128개 unit의 Dense layer가 input 받음
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    #10개 unit(최종 라벨의 개수)의 output layer: softmax로 확률 반환
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    
])

In [ ]:
#모델 compile
#optimizer = unit 내부 weight, bias를 변환시키는 알고리즘
#loss function = 예측한 값이 얼마나 실제와 동떨어졌는지 측정함
#metrics = 훈련과 테스트를 모니터하기 위함

model.compile(optimizer = 'adam',
             loss = tf.keras.losses.SparseCategoricalCrossentropy(),
             metrics=['accuracy'])

In [ ]:
#모델 Train
#1. train dataset에 미리 설정하기 - .repeat().shuffle().batch()
#batch개수(32개)가 끝난 이후에 optimizer가 weight, bias를 바꾸고, 훈련을 반복할 때마다
#데이터 순서를 랜덤으로 배정하도록 만듦

BATCH_SIZE = 32
train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.cache().batch(BATCH_SIZE)

In [ ]:
#2. Fit을 이용해 실제 훈련시키기!
#epoch = 반복 횟수

model.fit(train_dataset, epochs=10, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))


In [ ]:
#모델의 정확도 측정하기

test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples/32))

print('Accuracy on test dataset:', test_accuracy)

In [ ]:
#모델을 실제로 사용하기
# test_dataset에 있는 각 이미지에 대해, 0~9까지의 label이 될 확률을 각각 계산함 

for test_images, test_labels in test_dataset.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

#predictions라는 배열의 차원을 보여줌

predictions.shape

In [ ]:
#첫 번째 이미지의 prediction 보기
#여기서, softmax썼으니깐 10개 label이 맞을 확률을 10개 숫자로 갖는 배열로 반환함

predictions[0]

In [ ]:
#첫 번쨰 이미지의 prediction 배열 중 가장 높은 값을 갖는 아이템의 인덱스 반환

np.argmax(predictions[0])

#실제 테스트 세트의 첫 번째 이미지의 라벨값 반환

test_labels[0]


In [ ]:
#한 개의 이미지에 대한 예측하기

img = test_images[0]

#keras는 batch가 있어야지 예측이 가능하므로, 이미지 한 개여도 batch를 추가해야 함

img = np.array([img])

In [ ]:
#한 개의 이미지 예측 및 10개 아이템 가진 배열(확률값) 출력하기

predictions_single = model.predict(img)

printf(predictions_single)